<a href="https://colab.research.google.com/github/anusurabhi/Data-Science/blob/main/Loan%20Approval%20Health%20Check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data_info = pd.read_csv('sample_data/lending_club_loan_two.csv')

FileNotFoundError: ignored

In [ ]:
def mappingFunc(valueToChange):
    if(valueToChange == "Fully Paid"):
        return 1
    else:
        return 0

loan_repaid = map(mappingFunc, df['loan_status'])
loan_repaid = pd.DataFrame(loan_repaid)
df['loan_repaid'] = loan_repaid

df = df.drop('emp_title', axis=1)
df = df.drop('emp_length', axis=1)
total_acc_avg = df.groupby('total_acc').mean()['mort_acc']
def fill_mort_acc(total_acc, mort_acc):
    if np.isnan(mort_acc):
        return total_acc_avg[total_acc]
    else:
        return mort_acc
df['mort_acc'] = df.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)
df = df.dropna()
